# Example: Generating the force to displacement database using DatabaseSolver

### Import API and other packages

In [1]:
import numpy as np
import scipy as sp
import scipy.signal
from tqdm import tqdm
from matplotlib import pyplot as plt

from cnld.api import *
from cnld import fem, bem, util

### Define the CMUT geometry

In [ ]:
# start with 1 MHz preset geometry
geom = define.circular_cmut_1mhz_geometry()

# update preset with custom values
geom.thickness = 1e-6  # membrane thickness
geom.radius = 32e-6  # membrane radius
geom.isol_thickness = 100e-9  # thickness of isolation layer
geom.gap = 500e-9  # gap height
geom.electrode_r = 32e-6  # electrode radius
geom.damping_mode1 = 0  # first vibration mode to damp (mode 0 is fundamental mode)
geom.damping_mode2 = 5  # second vibration mode to damp (mode 5 is the next axisymmetric mode)
geom.damping_ratio1 = 0.1  # damping ratio of first mode
geom.damping_ratio2 = 0.1  # damping ratio of second mode

# define control domains (nr = ntheta = 1 lumps entire membrane as a single control domain)
geom.controldomain_nr = 1  # number of control domain divisions in radial direction
geom.controldomain_ntheta = 1  # number of control domain divisions in angular direction

# show properties of defined geometry
print(geom)

# run analysis of geometry to predict modes
analyze_geometry_modes(geom, refn=9, plot_modes=True)

### Define the CMUT array layout and generate grids (mesh)

In [ ]:
# use preset matrix layout
layout = define.matrix_layout(nx=2, ny=2, pitch_x=74e-6, pitch_y=74e-6)

# add defined geometry
layout.geometries.append(geom)

# generate control domains automatically
layout.controldomains = define.generate_controldomainlist(layout)

# generate FEM and BEM grids automatically with a refn = 9 (refinement level)
grids = grid.generate_grids_from_layout(layout, refn=9)

# plot the BEM grid which will show the layout
# warning: not advisable for large arrays!
grids.bem.plot()

### Generate the force to displacement database using DatabaseSolver

In [2]:
dbsolver = solve.DatabaseSolver()
dbsolver.file = 'circular_cmut_1mhz_hexagonal.db'
dbsolver.freq_interp = 4
dbsolver.overwrite = True
dbsolver.layout = layout
dbsolver.grids = grids
dbsolver.freqs = 0, 20e6, 100e3

dbsolver.solve()

Running: 100%|██████████| 200/200 [1:19:40<00:00, 23.90s/it]


In [28]:
from scipy import fftpack
import scipy as sp

def qfft(s, n=None, fs=1, axis=-1, full=False):

    ndim = s.ndim
    shape = s.shape
    nsample = shape[axis]

    if n is None:
        n = nsample

    if n > nsample:
        pw = [(0, 0)] * ndim
        pw[axis] = (0, n - nsample)
        s = np.pad(s, pw, mode='constant')
    elif n < nsample:
        s = np.take(s, range(n), axis=axis)

    ft = sp.fftpack.fft(s, axis=axis)
    freqs = sp.fftpack.fftfreq(n, 1 / fs)

    cutoff = (n + 1) // 2

    if full:
        return freqs, ft
    else:
        return freqs[:cutoff], np.take(ft, range(cutoff), axis=axis)


def qesd(s, n=None, fs=1, axis=-1, win=None):
    if win is None:
        win = np.ones(s.shape[axis])

    dims = np.ones(s.ndim, int)
    dims[axis] = -1
    win = win.reshape(dims)

    freqs, ft = qfft(s * win, n=n, fs=fs, full=False, axis=axis)
    return freqs, np.abs(ft)**2 * 2 / fs**2


fs = 1 / (tsolver.time[1] - tsolver.time[0])
x = np.mean(tsolver.displacement, axis=1)
freqs, esd = qesd(x, n=None, fs=fs)

plt.plot(freqs / 1e6, 10 * np.log10(esd / esd.max()))
plt.xlim(0, 50)

(0.0, 50.0)

In [6]:
from cnld import database, fem

freqs, ppfr = database.read_patch_to_patch_freq_resp('circular_cmut_1mhz_hexagonal.db')

fr = np.mean(np.sum(ppfr, axis=0), axis=0)
# fr = np.sum(ppfr, axis=0)[8, :]
plt.plot(freqs / 1e6, 10 * np.log10(np.abs(fr) / np.abs(fr).max()), '.-')
plt.xlim(0, 50)

(0.0, 50.0)